In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
# KFold는 학습 데이터 세트를 k개의 부분 데이터 세트(폴드)로 나눈 후, k-1개는 학습 데이터로 사용하고, 나머지 1개는 검증 데이터로 사용할 수 있도록 전체 학습 데이터 세트에서 인덱스를 나눠주는 역할을 한다.

In [ ]:
df = pd.read_csv("../../../Pytorch_sample/data/reg.csv", index_col=[0])
X = df.drop("Price", axis=1).to_numpy()
Y = df["Price"].to_numpy().reshape((-1, 1))

In [ ]:
# 텐서 데이터 만들기
# trainset은 교차 검증을 위해 나누기 때문에 미리 DataLoader를 정의하지 않는다.
class TensorData(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = self.y_data.shape[0]
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.7)
trainset = TensorData(X_train, Y_train)
testset = TensorData(X_test, Y_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

In [ ]:
# 모델 구축
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13, 50, bias = True)
        self.fc2 = nn.Linear(50, 30, bias = True)
        self.fc3 = nn.Linear(30, 1, bias = True)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(X)
        x = self.fc3(x)
        return x

In [ ]:
# 손실 함수와 교차 검증 정의
kfold = KFold(n_splits=3, shuffle=True) # 학습 데이터를 3개의 폴드로 나눠서 3겹 교차 검증을 수행한다.
criterion = nn.MSELoss()

In [ ]:
# 평가 함수 정의
def evaluation(dataloader):
    predictions = torch.tensor([], dtype=torch.float)
    actual = torch.tensor([], dtype=torch.float)

    with torch.no_grad():
        model.eval() # 평가를 할 떄는 반드시 .eval()을 사용해야 한다.
        for data in dataloader:
            inputs, values = data
            outputs = model(inputs)

            predictions = torch.cat((predictions, outputs), 0) # cat을 통해 예측값 누적
            actual = torch.cat((actual, values), 0) # cat을 통해 실제값 누적
    predictions = predictions.numpy()
    actual = actual.numpy()
    rmse = np.sqrt(mean_squard_error(predictions, actual))
    model.train()
    return rmse